In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

with tf.name_scope('placeholders'):
   x = tf.placeholder('float', [None, 1])
   y = tf.placeholder('float', [None, 1])

lay = {}
is_training = {}

is_training[0] = False
is_training[1] = False
is_training[2] = False
is_training[3] = False
dropout = 0.50 		# Dropout, probability to keep units

def random_dropout(is_dropout):
      for item in range(4):
         if is_dropout:
            is_training[item] = bool(np.round(np.random.rand(1))[0])
         else:
            is_training[item] = False 
 
with tf.name_scope('neural_network'):
   lay[0] = tf.contrib.layers.fully_connected(x,      30, activation_fn=tf.nn.tanh)
   lay[0] = tf.layers.dropout(lay[0], rate=dropout, training=is_training[0])
   
   lay[1] = tf.contrib.layers.fully_connected(lay[0], 20, activation_fn=tf.nn.tanh)
   lay[1] = tf.layers.dropout(lay[1], rate=dropout, training=is_training[1])

   lay[2] = tf.contrib.layers.fully_connected(lay[1], 10, activation_fn=tf.nn.tanh)
   lay[2] = tf.layers.dropout(lay[2], rate=dropout, training=is_training[2])

   lay[3] = tf.contrib.layers.fully_connected(lay[2], 20, activation_fn=tf.sigmoid)
   lay[3] = tf.layers.dropout(lay[3], rate=dropout, training=is_training[3])

   result = tf.contrib.layers.fully_connected(lay[3], 1, activation_fn=None)


   #loss = tf.reduce_sum(tf.nn.softmax(tf.nn.log_poisson_loss(result, y)))
   loss = tf.nn.l2_loss(result - y)
   #loss = tf.reduce_sum(tf.pow(tf.subtract(y, result), 2))
    
with tf.name_scope('optimizer'):
   #train_op = tf.train.MomentumOptimizer(0.01, 0.009).minimize(loss)
   train_op = tf.train.AdamOptimizer().minimize(loss)
   #train_op = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
    
with tf.Session() as sess:
   sess.run(tf.global_variables_initializer())

   plt.ion()
   fig = plt.figure()

   xpts_test = np.arange(0, 100, 0.1)
    
   xpts_train = np.random.rand(130) * 100 
   ypts_train = np.sin(xpts_train)

   cost_function_result = 1000.0
   training_epoch = -1

   while cost_function_result > 0.001:

      training_epoch = training_epoch + 1
         
      # compute cost function
      random_dropout(True)
      _, cost_function_result = sess.run([train_op, loss],
         feed_dict={x: xpts_train [:, None], y: ypts_train [:, None]})
        
      # evaluate model via drawing it's approximation
      random_dropout(False)
      if training_epoch % 10000 == 0:
         fig.clf()
         for plt_x, plt_y in zip(xpts_train, ypts_train):
            plt.plot(plt_x, plt_y, marker='o', markersize=3, color="red")

         ypts_test = sess.run(result, feed_dict={x: xpts_test[:, None]})
         for plt_x, plt_y in zip(xpts_test, ypts_test):
            plt.plot(plt_x, plt_y, marker='o', markersize=2, color="green")

         # draw to canvas and process window events            
         fig.canvas.draw()
         plt.pause(1)
         
      # print cost function
      if training_epoch % 100 == 0:
         print('iteration {}, loss={}'.format(training_epoch, cost_function_result))
